In [ ]:
import cv2 #perform image processing and computer vision tasks
import imutils #convenience functions to make basic image processing functions such as translation, rotation, resizing much more easier with OpenCV
import numpy as np
import keras #for implementation of neural network
import tensorflow as tf #makes implementation of cnn easy
from sudoku_solver import sol
from skimage.segmentation import clear_border
import easyocr #library for character recognition
class sudoku:

    def __init__(self,path):
        self.image=cv2.imread(path)
    #for converting the image into gray scale
    def grayscale(self):
        self.gray=cv2.cvtColor(self.image,cv2.COLOR_BGR2GRAY)
        return self.gray
    #appliying the gaussian blur on the image
    def GaussianBlur(self):
        self.blur=cv2.GaussianBlur(self.gray,(7,7),3)

        return self.blur


    #using adaptive Threshold because a paper may contain wide range of pixels
    def threshold(self):
        self.thresh=cv2.adaptiveThreshold(self.blur,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
        self.thresh=cv2.bitwise_not(self.thresh)

        return self.thresh
    #finding contours
    def contours(self):
        cnts = cv2.findContours(self.thresh.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

        cnts = imutils.grab_contours(cnts)

        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

        self.cnts=cnts

        return  cnts

    #getting the coordinates ROI
    def ROIpoints(self):
        self.puzzlecnt=[]
        for c in self.cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)

            if len(approx) == 4:
                x, y, w, h = cv2.boundingRect(approx)
                ROI = self.image[y:y + h, x:x + w]

                self.puzzlecnt.append(approx)
                break
        return self.puzzlecnt

    def order_points(self,pts):
        rect = np.zeros((4, 2), dtype="float32")
        # the top-left point will have the smallest sum, whereas
        # the bottom-right point will have the largest sum
        s = pts.sum(axis=1)
        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]
        # now, compute the difference between the points, the
        # top-right point will have the smallest difference,
        # whereas the bottom-left will have the largest difference
        diff = np.diff(pts, axis=1)
        rect[1] = pts[np.argmin(diff)]
        rect[3] = pts[np.argmax(diff)]
        # return the ordered coordinates
        return rect

    def tranformation(self):
        image=self.image
        points=np.array(self.puzzlecnt).reshape(4,2)
        rect = self.order_points(points)
        (tl, tr, br, bl) = rect
        widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
        widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))
        # compute the height of the new image, which will be the
        # maximum distance between the top-right and bottom-right
        # y-coordinates or the top-left and bottom-left y-coordinates
        heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
        heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))
        # now that we have the dimensions of the new image, construct
        # the set of destination points to obtain a "birds eye view",
        # (i.e. top-down view) of the image, again specifying points
        # in the top-left, top-right, bottom-right, and bottom-left
        # order
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]], dtype="float32")
        # compute the perspective transform matrix and then apply it
        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        # return the warped image
        return warped

    def Detect_Image(self,image):
        model = tf.keras.models.load_model('digit_model.h5')
        board = np.zeros((9, 9), dtype="int")
        # a Sudoku puzzle is a 9x9 grid (81 individual cells), so we can
        # infer the location of each cell by dividing the warped image
        # into a 9x9 grid
        stepX = image.shape[1] // 9
        stepY = image.shape[0] // 9
        # initialize a list to store the (x, y)-coordinates of each cell
        # location
        cellLocs = []
        js = 0
        for y in range(0, 9):
            # initialize the current list of cell locations
            row = []
            for x in range(0, 9):
                # compute the starting and ending (x, y)-coordinates of the
                # current cell
                startX = x * stepX
                startY = y * stepY
                endX = (x + 1) * stepX
                endY = (y + 1) * stepY
                # add the (x, y)-coordinates to our cell locations list
                row.append((startX, startY, endX, endY))
                image=cv2.rectangle(image,(startX,startY),(endX,endY),(255,34,233),2)
                #cutting the image into Region Of Image
                cut=image[startX:endX,startY:endY]

                cut=cv2.cvtColor(cut,cv2.COLOR_BGR2GRAY)
                cut=cv2.GaussianBlur(cut, (7, 7), 3)
                cut=cv2.adaptiveThreshold(cut, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
                cut=cv2.bitwise_not(cut)
                cut=clear_border(cut)

                edged = cv2.Canny(cut, 30, 200)
                counter=cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cnts = imutils.grab_contours(counter)

                cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

                cv2.drawContours(cut, cnts, -1, (0, 255, 0), 3)
                cut = cv2.resize(cut, (28, 28), interpolation=cv2.INTER_AREA)

                cellLocs.append(cut)
                cut=np.array(cut)
                cut=np.expand_dims(cut,axis=-1)
                cut=np.expand_dims(cut,axis=0)
                #print(cut.shape)
                print(cv2.contourArea(cnts[0]))
                if cv2.contourArea(cnts[0])>500:

                    ans=model.predict(cut).argmax()
                    ans=str(ans)
                    print(ans)
                    image = cv2.putText(image,ans, ((startX+endX)//2, (startY+endY)//2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255),2)

                js += 1
        print(js)

        return image
    def number_reader(self,image):
        reader = easyocr.Reader(['en'])
        result = reader.readtext(image)
        for i in result:
            image=cv2.rectangle(image,(i[0][0][0],i[0][0][1]),(i[0][2][0],i[0][2][1]),(255,34,233),2)
        return image
    def detect(self,image):

        stepX = image.shape[1] // 9
        stepY = image.shape[0] // 9
        # initialize a list to store the (x, y)-coordinates of each cell
        # location
        cellLocs = []

        reader = easyocr.Reader(['en'])
        result=[]
        sudoku=[[],[],[],[],[],[],[],[],[]]

        for y in range(0, 9):
            # initialize the current list of cell locations
            row = []
            js=0
            for x in range(0, 9):
                # compute the starting and ending (x, y)-coordinates of the
                # current cell
                startX = x * stepX
                startY = y * stepY
                endX = (x + 1) * stepX
                endY = (y + 1) * stepY
                image = cv2.rectangle(image, (startX, startY), (endX, endY), (255, 34, 233), 2)
                cut=image[startX:endX,startY:endY]
                result=[]
                try:
                    result = reader.readtext(cut)
                    print(result)
                except:
                    print("image not loaded correctly")
                # if j==20:
                #     cv2.imshow("cut",cut)
                #     cv2.waitKey(0)

                if result!=[]:
                    image = cv2.putText(image, result[0][-2], ((startY + endY) // 2,(startX + endX) // 2),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                    sudoku[js].append(int(result[0][-2]))

                else:
                    image = cv2.putText(image,'0', ((startY + endY) // 2, (startX + endX) // 2),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                    sudoku[js].append(0)


                js+=1

        print(sudoku)

        return image,sudoku

    def solve(self,sudoku):
        s=sol()

        s.print_board(sudoku)
        s.solve(sudoku)
        print("___________________")
        s.print_board(sudoku)

    def write_on_board(self,image,sudoku):
      stepX = image.shape[1] // 9
      stepY = image.shape[0] // 9
      for y in range(0, 9):
        row = []
        js=0
        for x in range(0, 9):
          startX = x * stepX
          startY = y * stepY
          endX = (x + 1) * stepX
          endY = (y + 1) * stepY
          image = cv2.putText(image, str(sudoku[x][y]),((startY + endY) // 2, (startX + endX) //2),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
      return image

# New Section

In [ ]:
pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.2/608.2 KB 46.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.72
    Uninstalling opencv-python-headless-4.7.0.72:
      Successfully uninstalled opencv-python-headless-4.7.0.72


In [ ]:
s=sudoku("city_times.jpeg")
# s = sudoku("sud.png")
image=s.grayscale()
cv2.imwrite("Grayscale.jpg", image)
image=s.GaussianBlur()
cv2.imwrite("GaussianBlur.jpg", image)
image=s.threshold()
cv2.imwrite("Threshold.jpg", image)
cnts=s.contours()
puzzlecnt=s.ROIpoints()
image=s.tranformation()
cv2.imwrite("Transformation.jpg", image)
image=cv2.resize(image,(900,900),interpolation=cv2.INTER_AREA)
image2=cv2.resize(image,(900,900),interpolation=cv2.INTER_AREA)
cv2.imwrite("Interpolation.jpg", image)

img,sudoku=s.detect(image)
s.solve(sudoku)
final = s.write_on_board(image2, sudoku)
cv2.imwrite("prefinal.jpg", img)
cv2.imwrite("final.jpg", final)

[([[39, 29], [79, 29], [79, 85], [39, 85]], '2', 0.9999828339359738)]
[]
[([[37, 23], [77, 23], [77, 77], [37, 77]], '9', 0.995717536344063)]
[([[43, 25], [79, 25], [79, 69], [43, 69]], '4', 0.9589750663995034)]
[]
[([[39, 17], [79, 17], [79, 71], [39, 71]], '5', 0.9999946355891645)]
[([[37, 21], [79, 21], [79, 73], [37, 73]], '6', 0.9980945136269384)]
[]
[([[35, 9], [77, 9], [77, 63], [35, 63]], '8', 0.9999834299773944)]
[]
[]
[([[33, 23], [75, 23], [75, 77], [33, 77]], '5', 0.999988794358174)]
[]
[([[35, 21], [75, 21], [75, 77], [35, 77]], '2', 0.9999710323524234)]
[]
[]
[([[35, 13], [75, 13], [75, 67], [35, 67]], '3', 0.9996267905422407)]
[]
[([[31, 29], [73, 29], [73, 83], [31, 83]], '6', 0.9926880045681088)]
[]
[([[31, 23], [73, 23], [73, 77], [31, 77]], '3', 0.9999868870211372)]
[([[31, 23], [71, 23], [71, 79], [31, 79]], '9', 0.9999980926522767)]
[]
[([[31, 17], [71, 17], [71, 71], [31, 71]], '7', 0.9999996423721633)]
[([[31, 21], [71, 21], [71, 75], [31, 75]], '2', 0.9999241843

True